# Problem-Specific Coordinate Generation for HyperNEAT Substrates

# Part 3: Neuroevolution

In this part, the created substrates are tested on the defined problem.

## Setup

### Imports

In [1]:
import os
import pickle
from collections import defaultdict

import wandb

from config import config

import jax
from evol_pipeline.brax_env import CustomBraxEnv
from evol_pipeline.custom_pipeline import CustomPipeline
from evol_pipeline.evol_algorithm import create_evol_algorithm

from utils.visualization import visualize_cppn, visualize_nn
from utils.utils import setup_folders_evolution, append_summary_row

The folder structure is checked and setup if necessary. A deep dictionary is defined and will hold the different substrates.

In [2]:
OUTPUT_DIR = config["experiment"]["output_dir_evolution"]
setup_folders_evolution(OUTPUT_DIR)

def deep_defaultdict():
    return defaultdict(deep_defaultdict)

### Setup Environment

[Brax environments](https://github.com/google/brax/tree/main/brax/envs) are used for this experiment through the [TensorNEAT wrapper](https://github.com/EMI-Group/tensorneat/tree/main/src/tensorneat/problem/rl).

In [3]:
env_name = config["experiment"]["env_name"]
env_problem = CustomBraxEnv(
    env_name=env_name,
    backend=config["environment"]["backend"],
    brax_args=config["environment"]["brax_args"],
    max_step=config["environment"]["max_step"],
    repeat_times=config["environment"]["repeat_times"],
    obs_normalization=config["environment"]["obs_normalization"],
    sample_episodes=config["environment"]["sample_episodes"],
)
obs_size = env_problem.input_shape[0]
act_size = env_problem.output_shape[0]

/home/andi/anaconda3/envs/jax/lib/python3.10/site-packages/brax/io/mjcf.py:480: UserWarning: Brax System, piplines and environments are not actively being maintained. Please see MJX for a well maintained JAX-based physics engine: https://github.com/google-deepmind/mujoco/tree/main/mjx. For a host of environments that use MJX, see: https://github.com/google-deepmind/mujoco_playground.
  warnings.warn(
2025-11-13 08:25:10.416269: W external/xla/xla/service/gpu/autotuning/dot_search_space.cc:200] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs?Working around this by using the full hints set instead.
2025-11-13 08:25:21.685618: W external/xla/xla/service/gpu/autotuning/dot_search_space.cc:200] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs?Working around this by usin

The substrates which were previously saved to disk are used for the neuroevolution process.

In [4]:
with open("substrates_d25.pkl", "rb") as f:
    substrates = pickle.load(f)

## Neuroevolution

Finally the substrates are ready to be used for neuroevolution with HyperNEAT. Weights and Biases (wandb) is used for logging and the CPPNs with highest fitness and their phenotypes are visualized. Coordinates are logged for further analysis.


In [5]:
for data_label, data_dict in substrates.items():
    for method_label, method_dict in data_dict.items():
        for feature_dims_label, feature_dims_dict in method_dict.items():

            active_substrate = substrates[data_label][method_label][feature_dims_label]["substrate"]
            evol_algorithm = create_evol_algorithm(substrate=active_substrate)
            substrate_dimensions = int(active_substrate.query_coors.shape[1]/2)
            initial_cppn_layers = config["algorithm"]["genome"]["cppn_init_hidden_layers"](active_substrate.query_coors.shape[1])

            run_id = f"{data_label}_{method_label}_{substrate_dimensions}d"

            wanbd_name = run_id
            wandb_tags = [config["substrate"]["hidden_layer_type"], env_name, data_label, method_label, f"{config['substrate']['hidden_depth']}_hl", f"{config['algorithm']['neat']['pop_size']}pop", f"{config['environment']['backend']}"]

            wandb.init(
                name=wanbd_name,
                project="substrates",
                tags=wandb_tags,
                config=config  
            )

            wandb.config.update(
                {
                    "substrate": {
                        "obs_size": obs_size,
                        "act_size": act_size,
                        "num_queries": active_substrate.query_coors.shape[0],
                        "query_dim": active_substrate.query_coors.shape[1],
                        },
                    "algorithm": {
                        "neat": {
                            "num_inputs": evol_algorithm.num_inputs,
                            },
                        "genome": {
                            "cppn_init_hidden_layers": initial_cppn_layers,
                            },
                        },
                },
            )
            
            genome_save_dir = os.path.join(OUTPUT_DIR, "genome", run_id)
            os.makedirs(genome_save_dir, exist_ok=True)

            pipeline = CustomPipeline(
                algorithm=evol_algorithm,
                problem=env_problem,
                seed=config["experiment"]["seed"],
                generation_limit=config["pipeline"]["generation_limit"],
                fitness_target=config["pipeline"]["fitness_target"],
                is_save=True,
                save_dir=genome_save_dir,
            )

            init_state = pipeline.setup()
            train_state, best_genome = pipeline.auto_run(state=init_state)

            print(f"\nTraining finished. Best fitness achieved: {pipeline.best_fitness}")

            wandb.finish()

            append_summary_row(
                path=f"{OUTPUT_DIR}/summary_results.csv",
                row={
                    "dimensionality": int(active_substrate.query_coors.shape[1] / 2),
                    "sampling": data_label,
                    "method": method_label,
                    "max_fitness": float(pipeline.best_fitness),
                },
            )

            # Uses the trained state for parameters, but overrides the key for reset/render. This way all videos start in the same state for comparison.
            k_show = jax.random.PRNGKey(config["experiment"]["seed"]+5)
            show_state = train_state.update(randkey=k_show)

            # Built-in show method to produce and save a video of the agent
            pipeline.show(
                state=show_state,
                best=best_genome,
                output_type="mp4",
                save_path=f"{OUTPUT_DIR}/video/agent_{data_label}_{method_label}_{feature_dims_label}.mp4",
            )

            # All coordinates are needed for visualization
            input_coors = substrates[data_label][method_label][feature_dims_label]["input_coors"]
            hidden_coors = substrates[data_label][method_label][feature_dims_label]["hidden_coors"]
            output_coors = substrates[data_label][method_label][feature_dims_label]["output_coors"]

            # Visualizes the CPPN
            visualize_cppn(
                pipeline=pipeline, 
                state=train_state, 
                save_path=f"{OUTPUT_DIR}/topology/cppn_{data_label}_{method_label}_{feature_dims_label}.svg"
            )
            
            # Visualizes a representation of the neural network in 2D space
            visualize_nn(
                pipeline=pipeline, 
                state=train_state, 
                save_path=f"{OUTPUT_DIR}/topology/nn_{data_label}_{method_label}_{feature_dims_label}.svg", 
                substrate=active_substrate, 
                input_coors=input_coors, 
                hidden_coors=hidden_coors, 
                output_coors=output_coors, 
                hidden_depth=config["substrate"]["hidden_depth"], 
                max_weight=config["algorithm"]["hyperneat"]["max_weight"], 
            )

wandb: Currently logged in as: wirkelzirkel (wirkelzirkel-iu) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


save to output_evolution/genome/trained_ica_25d
initializing
initializing finished
start compile
compile finished, cost time: 31.108128s
Generation: 1, Cost time: 4576.99ms
 	fitness: valid cnt: 500, max: 50.7569, min: -3963.1367, mean: -1318.2487, std: 1287.3062

	node counts: max: 54, min: 52, mean: 53.27
 	conn counts: max: 104, min: 51, mean: 101.97
 	species: 10, [24, 86, 28, 20, 18, 20, 17, 24, 15, 248]

Generation: 2, Cost time: 4458.11ms
 	fitness: valid cnt: 500, max: 54.6124, min: -3952.0212, mean: -429.4581, std: 895.8087

	node counts: max: 54, min: 51, mean: 53.16
 	conn counts: max: 105, min: 2, mean: 96.55
 	species: 10, [78, 80, 48, 40, 41, 1, 1, 1, 36, 174]

Generation: 3, Cost time: 4350.98ms
 	fitness: valid cnt: 500, max: 51.9041, min: -3953.1523, mean: -203.5170, std: 735.8945

	node counts: max: 55, min: 51, mean: 53.42
 	conn counts: max: 106, min: 2, mean: 94.76
 	species: 10, [8, 54, 56, 49, 67, 46, 38, 13, 9, 160]

Generation: 4, Cost time: 4379.24ms
 	fitness

compute_ms,▃▄▃▄▅▆█▆▆▆▆▁▁▁▁▁▂▂▂▃▃▃▃▄▄▄▄▄▃▄▄▄▄▄▄▄▄▄▄▄
compute_ms_pop,▁▁▂▃▃▆▇▆██▇▇▇▁▂▂▃▃▂▂▂▃▃▃▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄
fitness_max,▁▁▁▁▁▂▂▂▂▂▅▄▄▄▄▅▆▆▆▆▄▅▅▅▆▆▆▆▆▇▇▅▇██▇▇▇▇▇
fitness_mean,▃▂▅▁▄▆▃▅▆▆▆▄▇▆▅▇▇▅▆▇▇▅▇▅▆▆▅▅▆█▆▅▆▇▇▅██▇▅
fitness_min,▁▃▂▂▃▁▃▃▃▃▁▁▃▃▃▁▅▁▃▂█▄▂▅▃▅▃▁▁▁▃▃▃▁▁▂▃▃▃▁
fitness_std,█▂▃▂▂▃▃▂▂▂▁▂▂▃▂▂▁▁▁▂▂▂▂▂▂▂▁▁▁▁▁▂▁▂▂▂▂▃▂▂
generation,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▇▇██
num_species,█▇▇▇█▆▅▄▄▄▃▃▃▂▁▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
compute_ms,4590.15632
compute_ms_pop,9.18031
fitness_max,421.6051


2025-11-13 09:04:20.038484: W external/xla/xla/service/gpu/autotuning/dot_search_space.cc:200] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs?Working around this by using the full hints set instead.
2025-11-13 09:04:20.038500: W external/xla/xla/service/gpu/autotuning/dot_search_space.cc:200] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs?Working around this by using the full hints set instead.


Total reward:  472.0851


...

save to output_evolution/genome/no_data_random_coors_25d
initializing
initializing finished
start compile
compile finished, cost time: 30.300269s
Generation: 1, Cost time: 4542.63ms
 	fitness: valid cnt: 500, max: 50.8392, min: -3961.5825, mean: -1206.5063, std: 1292.1938

	node counts: max: 54, min: 52, mean: 53.27
 	conn counts: max: 104, min: 51, mean: 101.97
 	species: 10, [28, 86, 21, 26, 11, 8, 21, 17, 14, 268]

Generation: 2, Cost time: 4471.47ms
 	fitness: valid cnt: 500, max: 74.2467, min: -3949.5024, mean: -287.4281, std: 782.2709

	node counts: max: 55, min: 52, mean: 53.32
 	conn counts: max: 106, min: 51, mean: 96.41
 	species: 10, [75, 8, 1, 3, 1, 2, 61, 20, 13, 316]

Generation: 3, Cost time: 4438.58ms
 	fitness: valid cnt: 500, max: 92.1470, min: -3950.6538, mean: -174.8587, std: 750.3221

	node counts: max: 55, min: 52, mean: 53.74
 	conn counts: max: 107, min: 51, mean: 102.34
 	species: 10, [2, 78, 66, 19, 27, 32, 19, 1, 9, 247]

Generation: 4, Cost time: 4363.34ms
 

compute_ms,▂▁▂▂▂▄▄▄▄▄▅▄▅▅▆▇▇▇▇▇▇▇▇▇▇▇██████████████
compute_ms_pop,▁▁▁▂▂▃▃▃▄▃▄▄▅▅▅▆▇▇▇▇▇███████████████████
fitness_max,▁▂▄▄▄▄▅▄▄▄▄▄▄▆▅▇▇██▇█▇▇▆▇▆▇▇█▇█▇▇▇▇▇▇▇█▇
fitness_mean,▆▅▆▆█▄▅▂▁▃▃▆▅▇▅▅▆█▇█▄▄▆▆▆█▆▅▃▅▅▃▄▃▇▆▇▅▇▇
fitness_min,▁▁▂▁█▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
fitness_std,▁▁▃▄▂▃▁▃▃▄▄▃▅▃▄▅▅▃█▆▅▄▆▄▅▆▅█▇▅▅▆▆▄▆▄▆▄▆▅
generation,▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▅▆▇▇▇▇▇▇▇██
num_species,██████▅▂▁▁▃▃▃▄▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
compute_ms,4640.58638
compute_ms_pop,9.28117
fitness_max,624.75891


Total reward:  351.15945


/home/andi/anaconda3/envs/jax/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(


mp4 saved to:  output_evolution/video/agent_no_data_random_coors_24fd.mp4
Visualizing CPPN. Saving to output_evolution/topology/cppn_no_data_random_coors_24fd.svg.
Manually reconstructing the phenotype. A visual layout will be generated.
Substrate has 1792 potential connections.
Visualizing 1725 connections. Excluded loops. Weight threshold: 0.005
Visualization saved to: output_evolution/topology/nn_no_data_random_coors_24fd.svg


The isomap substrate was added after the other runs had already started, so this has to run seperately.

In [6]:
for data_label, data_dict in substrates.items():
    for feature_dims_label, feature_dims_dict in data_dict["isomap"].items():

        active_substrate = substrates[data_label]["isomap"][feature_dims_label]["substrate"]
        evol_algorithm = create_evol_algorithm(substrate=active_substrate)
        substrate_dimensions = int(active_substrate.query_coors.shape[1]/2)
        initial_cppn_layers = config["algorithm"]["genome"]["cppn_init_hidden_layers"](active_substrate.query_coors.shape[1])

        run_id = f"{data_label}_isomap_{substrate_dimensions}d"

        wanbd_name = run_id
        wandb_tags = [config["substrate"]["hidden_layer_type"], env_name, data_label, "isomap", f"{config['substrate']['hidden_depth']}_hl", f"{config['algorithm']['neat']['pop_size']}pop", f"{config['environment']['backend']}"]

        wandb.init(
            name=wanbd_name,
            project="substrates",
            tags=wandb_tags,
            config=config  
        )

        wandb.config.update(
            {
                "substrate": {
                    "obs_size": obs_size,
                    "act_size": act_size,
                    "num_queries": active_substrate.query_coors.shape[0],
                    "query_dim": active_substrate.query_coors.shape[1],
                    },
                "algorithm": {
                    "neat": {
                        "num_inputs": evol_algorithm.num_inputs,
                        },
                    "genome": {
                        "cppn_init_hidden_layers": initial_cppn_layers,
                        },
                    },
            },
        )
        
        genome_save_dir = os.path.join(OUTPUT_DIR, "genome", run_id)
        os.makedirs(genome_save_dir, exist_ok=True)

        pipeline = CustomPipeline(
            algorithm=evol_algorithm,
            problem=env_problem,
            seed=config["experiment"]["seed"],
            generation_limit=config["pipeline"]["generation_limit"],
            fitness_target=config["pipeline"]["fitness_target"],
            is_save=True,
            save_dir=genome_save_dir,
        )

        init_state = pipeline.setup()
        train_state, best_genome = pipeline.auto_run(state=init_state)

        print(f"\nTraining finished. Best fitness achieved: {pipeline.best_fitness}")

        wandb.finish()

        append_summary_row(
            path=f"{OUTPUT_DIR}/summary_results.csv",
            row={
                "dimensionality": int(active_substrate.query_coors.shape[1] / 2),
                "sampling": data_label,
                "method": "isomap",
                "max_fitness": float(pipeline.best_fitness),
            },
        )

        # Uses the trained state for parameters, but overrides the key for reset/render. This way all videos start in the same state for comparison.
        k_show = jax.random.PRNGKey(config["experiment"]["seed"]+5)
        show_state = train_state.update(randkey=k_show)

        # Built-in show method to produce and save a video of the agent
        pipeline.show(
            state=show_state,
            best=best_genome,
            output_type="mp4",
            save_path=f"{OUTPUT_DIR}/video/agent_{data_label}_isomap_{feature_dims_label}.mp4",
        )

        # All coordinates are needed for visualization
        input_coors = substrates[data_label]["isomap"][feature_dims_label]["input_coors"]
        hidden_coors = substrates[data_label]["isomap"][feature_dims_label]["hidden_coors"]
        output_coors = substrates[data_label]["isomap"][feature_dims_label]["output_coors"]

        # Visualizes the CPPN
        visualize_cppn(
            pipeline=pipeline, 
            state=train_state, 
            save_path=f"{OUTPUT_DIR}/topology/cppn_{data_label}_isomap_{feature_dims_label}.svg"
        )
        
        # Visualizes a representation of the neural network in 2D space
        visualize_nn(
            pipeline=pipeline, 
            state=train_state, 
            save_path=f"{OUTPUT_DIR}/topology/nn_{data_label}_isomap_{feature_dims_label}.svg", 
            substrate=active_substrate, 
            input_coors=input_coors, 
            hidden_coors=hidden_coors, 
            output_coors=output_coors, 
            hidden_depth=config["substrate"]["hidden_depth"], 
            max_weight=config["algorithm"]["hyperneat"]["max_weight"], 
        )